# CMA-ES Approach in finding Dwell Time Profiles from NETSEQ Data

## Problem

The NET-seq signal $S(x) = D(x) \cdot j(x)$, where $D(x)$ is the RNAP dwell time and $j(x)$ is the position-dependent flux. The TASEP simulation is a nonlinear forward model $F: D \to S$. We seek $D^*$ minimizing $\|F(D^*) - S_{\text{exp}}\|^2$.

## Why CMA-ES?

Richardson-Lucy iterative deconvolution fails because:
1. The forward model is nonlinear (exclusion, Rho termination feedback)
2. RL converges to a fixed point that is *worse* than the initial guess (see EXPLORATION.md)

CMA-ES (Covariance Matrix Adaptation Evolution Strategy) is a gradient-free evolutionary optimizer that:
- Does not require or approximate gradients
- Handles noisy fitness functions (rank-based selection, not value-based)
- Scales to ~1000D with sep-CMA-ES (diagonal covariance, O(n) complexity)
- Naturally explores the loss landscape without the self-defeating feedback that breaks RL

## Approach

- Optimize in log-space: $\theta(x) = \log D(x)$ to enforce $D(x) > 0$
- Use sep-CMA-ES (`CMA_diagonal=True`) for O(n) scaling to 1149 dimensions
- Initial guess: $D^0 = S_{\text{exp,norm}}$ (known near-optimal from RL analysis)
- Small $\sigma_0 = 0.1$ in log-space (~10% perturbation in D-space)
- Objective: $\text{MSE}(S_{\text{exp,norm}},\; S_{\text{sim,norm}}(D))$

In [1]:
try:
    import cma
    print(f"cma version: {cma.__version__}")
except ImportError:
    import subprocess, sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "cma"])
    import cma
    print(f"Installed cma version: {cma.__version__}")

import os
import time
import pickle
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

from netseq_tasep_fast import (
    netseq_tasep_fast,
    _load_gene_parameters,
)

cma version: 4.4.2


## Configuration

In [4]:
# --- Gene and simulation parameters ---
GENE = "insQ"
N_RUNS = 200          # TASEP runs per objective evaluation (noise/speed tradeoff)
N_WORKERS = None      # None = os.cpu_count()

# --- CMA-ES parameters ---
SIGMA0 = 0.1         # Initial step size in log-space (~10% perturbation in D-space)
MAX_GENERATIONS = 500 # Maximum CMA-ES generations
POPSIZE = None        # None = pycma default (4 + floor(3*ln(n)) = 25 for n=1149)

# --- Logging/checkpointing ---
CHECKPOINT_EVERY = 25      # Save checkpoint every N generations
CHECKPOINT_DIR = Path("cmaes_checkpoints")
DISPLAY_EVERY = 1          # Print progress every N generations

# --- Reproducibility ---
CMA_SEED = 67              # CMA-ES internal RNG seed
SIM_BASE_SEED = 10_000     # Base seed for TASEP simulations (incremented per eval)

# --- Resume from checkpoint (set to path to resume, or None to start fresh) ---
RESUME_FROM = None          # e.g., "cmaes_checkpoints/gen_0100.pkl"

In [3]:
# --- Load gene data ---
base_params = _load_gene_parameters(GENE)
S_exp_norm = base_params["RNAP_dwellTimeProfile"].copy()  # Already normalized to mean=1
gene_length = len(S_exp_norm)

print(f"Gene: {GENE}")
print(f"Gene length: {gene_length} bp")
print(f"KRutLoading: {base_params['KRutLoading']}")
print(f"kRiboLoading: {base_params['kRiboLoading']:.4f}")

# Initial guess in log-space
D_init = S_exp_norm.copy()
theta_init = np.log(D_init)

print(f"\nlog(D_init) stats: mean={theta_init.mean():.4f}, std={theta_init.std():.4f}")
print(f"D_init stats: mean={D_init.mean():.4f}, min={D_init.min():.4f}, max={D_init.max():.4f}")

Gene: insQ
Gene length: 1149 bp
KRutLoading: 0.13
kRiboLoading: 0.0220

log(D_init) stats: mean=-0.7579, std=1.0240
D_init stats: mean=1.0000, min=0.1948, max=58.0638


## Objective Function

CMA-ES optimizes $\theta(x) = \log D(x)$. The objective:
1. Exponentiates: $D = \exp(\theta)$
2. Normalizes: $D_{\text{norm}} = D / \text{mean}(D)$
3. Runs TASEP forward model (averaged over `N_RUNS` stochastic realizations)
4. Normalizes output: $S_{\text{sim,norm}} = S_{\text{sim}} / \text{mean}(S_{\text{sim}})$
5. Returns: $\text{MSE} = \frac{1}{L}\sum_x (S_{\text{exp}}(x) - S_{\text{sim}}(x))^2$

Each evaluation uses a unique deterministic seed to avoid overfitting to a particular noise realization.

In [5]:
def run_simulations_with_dwell(dwell_profile, base_params, n_runs, base_seed, n_workers=None):
    """Run n_runs TASEP simulations with a custom dwell profile and return mean NETseq_sum."""
    params = dict(base_params)
    params["RNAP_dwellTimeProfile"] = dwell_profile

    seeds = [base_seed + i for i in range(n_runs)]
    if n_workers is None:
        n_workers = os.cpu_count() or 1

    def worker(s):
        result = netseq_tasep_fast(params, seed=s)
        return np.asarray(result["NETseq_sum"], dtype=float)

    if n_workers <= 1:
        outputs = [worker(s) for s in seeds]
    else:
        with ThreadPoolExecutor(max_workers=n_workers) as executor:
            outputs = list(executor.map(worker, seeds))

    netseq_total = np.zeros_like(outputs[0], dtype=float)
    for output in outputs:
        netseq_total += output
    return netseq_total / float(n_runs)


# Global evaluation counter for deterministic seed management
_eval_counter = 0

def objective(theta):
    """CMA-ES objective: MSE between S_exp and TASEP(exp(theta))."""
    global _eval_counter
    _eval_counter += 1

    # Exponentiate to get D > 0, normalize to mean=1
    D = np.exp(theta)
    D_norm = D / np.mean(D)

    # Run TASEP forward model with unique seed
    eval_seed = SIM_BASE_SEED + _eval_counter * N_RUNS
    S_sim = run_simulations_with_dwell(D_norm, base_params, N_RUNS, eval_seed, N_WORKERS)

    # Normalize simulated signal
    S_sim_mean = np.mean(S_sim)
    if S_sim_mean > 0:
        S_sim_norm = S_sim / S_sim_mean
    else:
        return 1e6  # Degenerate: no RNAPs survived

    return np.mean((S_exp_norm - S_sim_norm) ** 2)

## Baseline Evaluation

Evaluate the initial guess $D^0 = S_{\text{exp,norm}}$ to establish starting fitness. This should match the RL iter-0 residual from `iterative_deconvolution.ipynb` (~3.48 RMS).

In [7]:
t0 = time.time()
baseline_mse = objective(theta_init)
t_baseline = time.time() - t0

baseline_rms = np.sqrt(baseline_mse)

popsize_est = POPSIZE or (4 + int(3 * np.log(gene_length)))

print(f"Baseline MSE (D=S_exp_norm): {baseline_mse:.6f}")
print(f"Baseline RMS residual:       {baseline_rms:.4f}")
print(f"Evaluation time:             {t_baseline:.2f}s")
print(f"\nPopulation size: {popsize_est}")
print(f"Estimated time per generation: ~{t_baseline * popsize_est:.1f}s")
print(f"Estimated total for {MAX_GENERATIONS} generations: ~{t_baseline * popsize_est * MAX_GENERATIONS / 3600:.1f} hours")

Baseline MSE (D=S_exp_norm): 12.327057
Baseline RMS residual:       3.5110
Evaluation time:             1.15s

Population size: 25
Estimated time per generation: ~28.8s
Estimated total for 500 generations: ~4.0 hours


## CMA-ES Optimization

Using sep-CMA-ES (`CMA_diagonal=True`) for O(n) time and space complexity. At n=1149, full CMA-ES would require an O($n^3$) eigendecomposition per generation. Sep-CMA-ES stores only the diagonal and runs in O(n).

In [ ]:
# Create checkpoint directory
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)

# CMA-ES options
opts = {
    'CMA_diagonal': True,       # sep-CMA-ES: diagonal covariance, O(n)
    'seed': CMA_SEED,
    'maxiter': MAX_GENERATIONS,
    'verb_disp': 0,             # Suppress pycma's own printing
    'verb_log': 0,              # Disable pycma file logging (we do our own)
    'tolfun': 1e-8,             # Very tight: let it run until maxiter
    'tolx': 1e-8,               # Very tight: let it run until maxiter
}
if POPSIZE is not None:
    opts['popsize'] = POPSIZE

# Initialize or resume
if RESUME_FROM is not None:
    with open(RESUME_FROM, 'rb') as f:
        checkpoint = pickle.load(f)
    es = checkpoint['es']
    history = checkpoint['history']
    _eval_counter = checkpoint['eval_counter']
    print(f"Resumed from {RESUME_FROM}, generation {len(history['gen'])}")
else:
    es = cma.CMAEvolutionStrategy(theta_init, SIGMA0, opts)
    history = {
        'gen': [],
        'f_best': [],
        'f_median': [],
        'sigma': [],
        'elapsed': [],
        'theta_best': None,
    }
    _eval_counter = 0
    print(f"Starting fresh CMA-ES optimization")

print(f"  Dimension: {es.N}")
print(f"  Population size: {es.popsize}")
print(f"  sigma0: {SIGMA0}")
print(f"  n_runs per eval: {N_RUNS}")

In [ ]:
t_start = time.time()
gen_count = len(history['gen'])

try:
    while not es.stop():
        # Sample population
        candidates = es.ask()

        # Evaluate each candidate
        fitnesses = [objective(x) for x in candidates]

        # Update distribution
        es.tell(candidates, fitnesses)

        gen_count += 1
        elapsed = time.time() - t_start

        # Log
        f_best = es.result.fbest
        f_median = np.median(fitnesses)
        sigma = es.sigma

        history['gen'].append(gen_count)
        history['f_best'].append(f_best)
        history['f_median'].append(f_median)
        history['sigma'].append(sigma)
        history['elapsed'].append(elapsed)
        history['theta_best'] = es.result.xbest.copy()

        if gen_count % DISPLAY_EVERY == 0:
            eta = elapsed / gen_count * (MAX_GENERATIONS - gen_count)
            print(f"  gen {gen_count:4d} | f_best={f_best:.6f} | f_med={f_median:.6f} | "
                  f"sigma={sigma:.4f} | {elapsed:.0f}s | ETA {eta:.0f}s")

        # Checkpoint
        if gen_count % CHECKPOINT_EVERY == 0:
            ckpt_path = CHECKPOINT_DIR / f"gen_{gen_count:04d}.pkl"
            with open(ckpt_path, 'wb') as f:
                pickle.dump({
                    'es': es,
                    'history': history,
                    'eval_counter': _eval_counter,
                }, f)
            print(f"  >> Checkpoint saved: {ckpt_path}")

except KeyboardInterrupt:
    print(f"\n  Interrupted at generation {gen_count}. Saving checkpoint...")
    ckpt_path = CHECKPOINT_DIR / f"gen_{gen_count:04d}_interrupted.pkl"
    with open(ckpt_path, 'wb') as f:
        pickle.dump({
            'es': es,
            'history': history,
            'eval_counter': _eval_counter,
        }, f)
    print(f"  >> Checkpoint saved: {ckpt_path}")

total_time = time.time() - t_start
stop_conditions = es.stop()
print(f"\nDone in {total_time:.1f}s ({gen_count} generations)")
print(f"Stop conditions: {stop_conditions}")
print(f"Best MSE: {es.result.fbest:.6f} (RMS: {np.sqrt(es.result.fbest):.4f})")
print(f"Baseline MSE was: {baseline_mse:.6f} (RMS: {baseline_rms:.4f})")
print(f"Total evaluations: {_eval_counter}")

In [ ]:
# Save final state
final_path = CHECKPOINT_DIR / "final.pkl"
with open(final_path, 'wb') as f:
    pickle.dump({
        'es': es,
        'history': history,
        'eval_counter': _eval_counter,
        'D_best': np.exp(es.result.xbest) / np.mean(np.exp(es.result.xbest)),
        'S_exp_norm': S_exp_norm,
        'gene': GENE,
    }, f)
print(f"Final state saved to {final_path}")

## Convergence Analysis

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

gens = np.array(history['gen'])

# Best objective
axes[0].semilogy(gens, history['f_best'], '-', color='steelblue', lw=1)
axes[0].axhline(baseline_mse, color='gray', ls='--', lw=1, label=f'Baseline: {baseline_mse:.4f}')
axes[0].set_xlabel('Generation')
axes[0].set_ylabel('Best MSE (log scale)')
axes[0].set_title('Objective convergence')
axes[0].legend(fontsize=8)

# Best vs median
axes[1].semilogy(gens, history['f_median'], '-', color='tab:orange', lw=0.8, alpha=0.7, label='Median')
axes[1].semilogy(gens, history['f_best'], '-', color='steelblue', lw=1, label='Best')
axes[1].set_xlabel('Generation')
axes[1].set_ylabel('MSE (log scale)')
axes[1].set_title('Best vs median fitness')
axes[1].legend(fontsize=8)

# Sigma
axes[2].plot(gens, history['sigma'], '-', color='tab:green', lw=1)
axes[2].set_xlabel('Generation')
axes[2].set_ylabel('sigma')
axes[2].set_title('Step-size adaptation')

plt.tight_layout()
plt.show()

## Results: High-Fidelity Evaluation

Re-evaluate the best $D^*$ and the baseline with 300 runs for a clean comparison.

In [ ]:
# Extract best D*
theta_best = es.result.xbest
D_best = np.exp(theta_best)
D_best_norm = D_best / np.mean(D_best)

# High-fidelity evaluation (300 runs)
print("Running high-fidelity evaluation of D* (300 runs)...")
t0 = time.time()
S_sim_best = run_simulations_with_dwell(D_best_norm, base_params, 300, 99999, N_WORKERS)
S_sim_best_norm = S_sim_best / np.mean(S_sim_best)
print(f"  Done in {time.time() - t0:.1f}s")

print("Running high-fidelity baseline (300 runs)...")
t0 = time.time()
S_sim_baseline = run_simulations_with_dwell(S_exp_norm, base_params, 300, 88888, N_WORKERS)
S_sim_baseline_norm = S_sim_baseline / np.mean(S_sim_baseline)
print(f"  Done in {time.time() - t0:.1f}s")

mse_best = np.mean((S_exp_norm - S_sim_best_norm) ** 2)
mse_baseline = np.mean((S_exp_norm - S_sim_baseline_norm) ** 2)

print(f"\nHigh-fidelity MSE comparison:")
print(f"  Baseline (D=S_exp):  MSE={mse_baseline:.6f}, RMS={np.sqrt(mse_baseline):.4f}")
print(f"  CMA-ES optimized D*: MSE={mse_best:.6f}, RMS={np.sqrt(mse_best):.4f}")
if mse_baseline > 0:
    print(f"  Improvement: {(1 - mse_best / mse_baseline) * 100:.1f}%")

In [ ]:
pos = np.arange(1, gene_length + 1)

fig, axes = plt.subplots(3, 1, figsize=(14, 10), gridspec_kw={'height_ratios': [3, 3, 1]})

# S_exp vs S_sim(D*) vs S_sim(baseline)
axes[0].plot(pos, S_exp_norm, lw=0.6, color='black', label='S_exp (normalized)', alpha=0.8)
axes[0].plot(pos, S_sim_best_norm, lw=0.6, color='tab:blue', label='S_sim(D*) CMA-ES', alpha=0.8)
axes[0].plot(pos, S_sim_baseline_norm, lw=0.6, color='tab:red', label='S_sim(D=S_exp) baseline', alpha=0.5)
axes[0].set_ylabel('Normalized signal')
axes[0].set_title(f'{GENE}: Experimental vs Simulated NET-seq signal')
axes[0].legend(fontsize=9)

# D* vs S_exp
axes[1].plot(pos, S_exp_norm, lw=0.5, color='black', label='S_exp (naive D)', alpha=0.6)
axes[1].plot(pos, D_best_norm, lw=0.5, color='tab:blue', label='D* (CMA-ES optimized)')
axes[1].set_ylabel('Normalized dwell time')
axes[1].set_title('Dwell time: naive vs optimized')
axes[1].legend(fontsize=9)

# Point-wise residual
axes[2].plot(pos, S_exp_norm - S_sim_best_norm, lw=0.5, color='tab:blue', alpha=0.7, label='CMA-ES')
axes[2].plot(pos, S_exp_norm - S_sim_baseline_norm, lw=0.5, color='tab:red', alpha=0.4, label='Baseline')
axes[2].axhline(0, color='gray', ls='-', lw=0.5)
axes[2].set_xlabel('Position (bp)')
axes[2].set_ylabel('Residual')
axes[2].set_title('Point-wise residual (S_exp - S_sim)')
axes[2].legend(fontsize=9)

plt.tight_layout()
plt.show()

## Implied Flux Profile

If $S(x) = D(x) \cdot j(x)$ and we have $D^*$, then the implied flux is $j^*(x) = S_{\text{sim}}^*(x) / D^*(x)$.

In [ ]:
# Implied flux
j_implied = S_sim_best / np.maximum(D_best_norm, 1e-6)
j_implied_norm = j_implied / np.max(j_implied[:100])  # Normalize to ~1 at 5' end

# Analytical flux shape for comparison (Eq. M3)
ell_0 = 35 + 50  # ell_RNAP + ell_min = 85 bp
x = np.arange(1, gene_length + 1, dtype=float)
j_analytical = np.ones(gene_length)
mask = x > ell_0
j_analytical[mask] = np.exp(-base_params['KRutLoading'] / (2 * gene_length) * (x[mask] - ell_0) ** 2)

fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(pos, j_implied_norm, lw=0.8, color='tab:blue', label='j*(x) implied from CMA-ES')
ax.plot(pos, j_analytical, lw=1.5, color='tab:red', ls='--', label='j_analytical (Eq. M3, Gaussian approx)')
ax.set_xlabel('Position (bp)')
ax.set_ylabel('Flux j(x) / j(0)')
ax.set_title(f'{GENE}: Implied RNAP flux profile')
ax.legend()
plt.tight_layout()
plt.show()

## Correction Factor: D* / S_exp

The ratio $D^*(x) / S_{\text{exp}}(x)$ reveals the systematic correction CMA-ES applied. Under constant flux this ratio would be 1 everywhere. Deviations reveal flux contamination in the naive estimate.

In [ ]:
correction = D_best_norm / np.maximum(S_exp_norm, 1e-6)

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

axes[0].plot(pos, correction, lw=0.6, color='tab:purple')
axes[0].axhline(1.0, color='gray', ls='--', lw=0.5)
axes[0].set_xlabel('Position (bp)')
axes[0].set_ylabel('D* / S_exp')
axes[0].set_title('Correction factor (D* / S_exp)')

# Smoothed version
window = 25
correction_smooth = np.convolve(correction, np.ones(window) / window, mode='same')
axes[1].plot(pos, correction_smooth, lw=1.5, color='tab:purple', label=f'{window}-bp rolling mean')
axes[1].plot(pos, 1.0 / j_analytical, lw=1.5, color='tab:red', ls='--', label='1/j_analytical (predicted)')
axes[1].axhline(1.0, color='gray', ls='--', lw=0.5)
axes[1].set_xlabel('Position (bp)')
axes[1].set_ylabel('D* / S_exp (smoothed)')
axes[1].set_title('Smoothed correction vs analytical prediction')
axes[1].legend()

plt.tight_layout()
plt.show()